In [10]:
import os
import cv2
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mediapipe as mp

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(dataset_dir):
    X = []
    y = []
    
    # List of class labels (assuming folder names are the labels)
    class_labels = sorted(os.listdir(dataset_dir))
    
    for label in class_labels:
        label_dir = os.path.join(dataset_dir, label)
        if not os.path.isdir(label_dir):
            continue
        
        for img_filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_filename)
            image = cv2.imread(img_path)
            
            if image is None:
                continue
            
            # Preprocess image (adjust based on your preprocessing function)
            processed_image = image_processed(image)
            
            X.append(processed_image)
            y.append(int(label))  # Convert the folder name to an integer label
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

def image_processed(hand_img):
    img_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)
    img_flip = cv2.flip(img_rgb, 1)
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)
    output = hands.process(img_flip)
    hands.close()
    try:
        data = output.multi_hand_landmarks[0]
        data = str(data).strip().split('\n')
        garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']
        without_garbage = [i for i in data if i not in garbage]
        clean = [float(i[2:].strip()) for i in without_garbage]
        return clean
    except:
        return np.zeros([1,63], dtype=int)[0]

# Load your trained model
with open('model_mckiler.pkl', 'rb') as f:
    svm = pickle.load(f)

# Load test data
dataset_dir = r"C:\Users\hfhon\OneDrive\Desktop\train_model\testing_dataset"
X_test, y_test = load_dataset(dataset_dir)

# Evaluate the model
y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


C:\Users\hfhon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


ValueError: invalid literal for int() with base 10: 'yrnyung'